# Gyrus Overview (`gyrus==0.1.0`)

Recursively generate large-scale Nengo models using NumPy semantics.

## Cheat Sheet (API Summary)

**Operator methods** consume an operator as their first argument, and produce some other
operator (which typically depends on the given operator as input). They are also
vectorized to work element-wise across "folds" (i.e., N-D arrays of operators),
analogous to "ufuncs" in NumPy. Each method can be called either as a function, or as a
method on the operator, in the same way that `np.sum(arr)` is equivalent to `arr.sum()`:
 - `apply` - Operator that applies a function to each output ideally using a Node.
 - `configure` - Operator that applies configuration settings to all downstream
operators.
 - `convolve` - Operator that approximates the circular convolution using Product
networks.
 - `filter` - Operator that applies a synaptic filter to each output.
 - `decode` - Operator that approximates a function of each output using an Ensemble.
 - `integrate` - Operator that solves $\dot{x} = u + \text{integrand}(x)$ using Euler's
method.
 - `transform` - Operator that applies a single transform to every output.
 - `lti` - Operator that solves $\dot{x} = A.\text{state}(x) + B.u$. where $A$, $B$ =
system.
 - `multiply` - Operator that approximates an element-wise product using a Product
network.
 - `neurons` - Operator that linearly projects to a layer of neurons.
 - `slice` - Operator that slices each output using Nengo's object slicing.
 - `unbundle` - Operator that splits each output into a Fold of one-dimensional outputs.

**Operator attributes** can be accessed from any operator or fold:
 - `input_ops` - Tuple of operators that the operator depends on as inputs.
 - `label` - Returns the label of the operator's output as it appears in Nengo GUI.
 - `ndim` - Returns the dimensionality of the underlying NumPy array (or `0`).
 - `shape` - Returns the shape of the underlying NumPy array (or `()`).
 - `size_out` - Returns the `size_out` of each element of the underlying NumPy array.

**Fold methods** manipulate folds, which are a special kind of operator equivalent to an
`np.ndarray` where each element of the array is another operator. These methods are like
operator methods, but consume and produce folds:
 - `bundle` - Operator that joins all of the outputs along a given axis into a single
output.
 - `reduce_transform` - Operator that sums together transforms applied to each output
along an axis.

**NumPy array functions** are like Fold methods, but in actuality are NumPy array
functions applied to the fold's underlying array:
 - Functional programming routines: `np.apply_along_axis`.
 - Math routines: `np.dot`, `np.mean`, `np.outer`, `np.prod`, `np.sum`.
 - Changing array shape: `np.reshape`, `np.ravel`.
 - Transpose-like operations: `np.moveaxis` `np.rollaxis`, `np.swapaxes`,
`np.transpose`.
 - Changing number of dimensions: `np.atleast_1d`, `np.atleast_2d`, `np.atleast_3d`,
`np.broadcast_to`, `np.broadcast_arrays`, `np.expand_dims`, `np.squeeze`.
 - Joining arrays: `np.concatenate`, `np.stack`, `np.block`, `np.vstack`, `np.hstack`,
`np.dstack`, `np.column_stack`.
 - Splitting arrays: `np.split`, `np.array_split`, `np.dsplit`, `np.hsplit`,
`np.vsplit`.
 - Tiling arrays: `np.repeat`, `np.tile`.
 - Rearranging elements: `np.flip`, `np.fliplr`, `np.flipud`, `np.reshape`, `np.roll`,
`np.rot90`.

**NumPy array methods** are like NumPy array functions, but are only accessible as
methods on a given fold:
 - `flatten` - Equivalent to `np.ndarray.flatten`.
 - `T` - Equivalent to `np.ndarray.T`.
 - `__getitem__` - Equivalent to `ndarray.__getitem__`.

**Fold attributes** can be accessed from any fold:
 - `array` - Returns the Fold as a read-only NumPy array.

**Operator input functions** create roots of the operator graph (i.e., operators without
any input operators):
 - `broadcast_scalar` - Operator that creates a scalar Node and transforms it to match
some shape.
 - `stimuli` - Operator that supplies input data given Nengo objects or Node outputs.
(Vectorized version of `stimulus`.)
 - `stimulus` - Operator that supplies input data given a Nengo object or a Node output.

**Top-level functions** that don't fit under any other category:
 - `asoperator` - Returns x as a single Operator if possible, otherwise as a Fold.
 - `fold` - Return a fold from an iterable of operators or folds.

**Extensions** allow users to easily define their own Gyrus operators and register them
as array functions and ufuncs:
 - `register_method` - Register a function as a method that takes self as its first
argument.
 - `register_ufunc` - Register a function as a NumPy ufunc handler for this type.
 - `vectorize` - Dynamically creates a vectorized operator type with the given
implementation.

**Python dunder operators** overload the behaviour of common Python operations:
 - add (`+`)
 - subtract (`-`)
 - multiply (`*`)
 - true divide (`/`)
 - power (`**`)
 - matmul (`@`)
 - slicing (`[...]`) at the Nengo Node level.

**NumPy ufuncs** approximate the behaviour of common NumPy ufuncs using the `decode`
operator:
 - `cos`
 - `sin`
 - `tanh`
 - `square`

**Operator outputs** are methods attached to each operator that allow the user to build
and simulate the Nengo model that produces the output(s) of the given operator:
 - `make` - Generates operator's graph within the current Nengo network context.
(Interoperable with existing Nengo networks.)
 - `probe` - Returns a function that consumes a simulator and produces probe data.
(Typically used in conjunction with `make`.)
 - `run` - Generates a Nengo network, then probes, simulates, and returns its data.
(Convenience wrapper around `make` and `probe`.)

**Optional operators methods** require installation with the extra, `[optional]`, and
give access to the following operator methods:
 - `layer` - Operator that applies a `nengo_dl.Layer` to each output.
 - `tensor_node` - Operator that applies a `nengo_dl.TensorNode` to each output.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
import nengo_dl
from nengo_gui.ipython import InlineGUI

import numpy as np
from scipy.special import legendre
import seaborn as sns

import gyrus

### Example #1 - Communication Channel

The obligatory example of encoding a scalar into a spiking ensemble and decoding out a
filtered version of the identity function.

Note: `decode` accepts a `function` parameter which defaults to the identity, and an
`n_neurons` parameter which currently defaults to 100.

In [ ]:
@gyrus.stimulus
def stim(t):
    return np.sin(2 * np.pi * t)


out = stim.decode().filter(5e-3).run(1)

In [ ]:
plt.figure()
plt.title("Example #1 - Communication Channel")
plt.plot(out)
plt.xlabel("Time-step")
plt.show()

The fact that operators are vectorized comes with some pleasantly surprising
consequences, such as the ability to take a signal and filter it with a number of
distinct synapses to produce a parallel fold:

In [ ]:
synapses = [0.005, 0.05, 0.1]
out = np.asarray(stim.decode().filter(synapses).run(1))
out.shape

In [ ]:
plt.figure()
plt.title("Example #1 - Communication Channel (Vectorized Filtering)")
for i, y_i in enumerate(out):
    plt.plot(y_i, label=fr"$\tau$ = {synapses[i]}s")
plt.xlabel("Time-step")
plt.legend()
plt.show()

### Example #2 - Ensemble Array

Ensemble arrays are 'trivial' in Gyrus – just add another dimension to the underlying
NumPy array. This naturally generalizes to higher-dimensional ndarrays of arbitrary
shape (a.k.a., tensors), which serves to overcome Nengo's limitation of representing
only scalars or vectors, while still being just Nengo under the hood.

Note: The `bundle(axis=0)` method changes the output shape from `(16, 1000, 1)` to
`(1000, 16)`.

In [ ]:
d = 16
stim = gyrus.stimuli(np.linspace(-1, 1, d))
assert stim.shape == (d,)

y = stim.decode().filter(1e-2).bundle()
assert y.size_out == d

out = np.asarray(y.run(1))
out.shape  # (outputs, time, size_out)

In [ ]:
plt.figure()
plt.title(f"Example #2 - Ensemble Array (d={d})")
plt.plot(out)
plt.xlabel("Time-step")
plt.show()

### Example #3 - Nonlinear Functions

Numeric operations are overloaded to automatically use neurons for each nonlinear
computation (e.g., `a ** 2` and `(...) * b`) and transforms for each linear computation
(e.g., `1 - ...` and `2 * b`).

In [ ]:
input_functions = [
    lambda t: 2 * t - 1,
    lambda t: 1 - 2 * t,
]


def f(a, b, tau=1e-2):  # like a subnetwork
    return ((1 - a ** 2).filter(tau) * (2 * b)).filter(tau)


x = gyrus.stimuli(input_functions)
y = f(*x)
out = np.asarray(y.run(1))

In [ ]:
plt.figure()
plt.title("Example #3 - Nonlinear Functions")
plt.plot(out)
plt.xlabel("Time-step")
plt.show()

We can make it more accurate by configuring the number of neurons used by all operators
downstream of `a` and/or `b`. Configuration follows natural precedence rules (first
left-to-right, then upstream).

In [ ]:
y = f(*x.configure(n_neurons=1000))
out = np.asarray(y.run(1))

In [ ]:
plt.figure()
plt.title("Example #3 - Nonlinear Functions (More Neurons)")
plt.plot(out)
plt.xlabel("Time-step")
plt.show()

### Example #4 - Building a Gyrus operator into a Nengo network

We now show how to make a Gyrus operator explicitly inside of our own `nengo.Network()`
definition using the `make()` method, and then use the `stimulus` function to create
Node inputs to the Gyrus operator.

In general, this pattern enables full interoperability with Nengo: `stimulus(node)` can
be used to create Gyrus inputs from Nengo nodes, and the output(s) of `make()` are
Nengo nodes that can be connected from just like any other Nengo object. This gives
users the ability to interface networks generated using Gyrus with existing Nengo
networks, and therefore a way to dig down into certain implementation details at the
level of Nengo and reuse existing Nengo code.

For sake of example, we remake the same network from example #3, but get the ideal
output (i.e., don't approximate the nonlinearities with neurons). To do so, we use
Nengo's `model.config` to set `neuron_type=nengo.Direct()` for all ensembles. An
equivalent approach would be to invoke `.configure(neuron_type=nengo.Direct())` on
either of the two `Stimulus` operators.

Note: `p = gyrus.probe((...).make())` and then `p(sim)` is an equivalent way to do the
probing, but has the added feature of being able to recursively probe entire folds.

In [ ]:
with nengo.Network() as model:
    model.config[nengo.Ensemble].neuron_type = nengo.Direct()

    a = nengo.Node(input_functions[0])
    b = nengo.Node(input_functions[1])

    out = f(gyrus.stimulus(a), gyrus.stimulus(b)).make()

    p = nengo.Probe(out)

with nengo.Simulator(model) as sim:
    sim.run(1)

In [ ]:
plt.figure()
plt.title("Example #4 - Building a Gyrus operator into a Nengo network")
plt.plot(sim.data[p])
plt.xlabel("Time-step")
plt.show()

We can also use the Nengo GUI to visualize the generated Nengo model. The "Decode"
ensemble belongs to the `a ** 2` operator and the "Multiply" subnetwork implements the
product with `b`. Nodes in between are responsible for linear transformations, and an
additional node is automatically created to supply the constant `1` inside of `f`.

In [ ]:
InlineGUI(model)

### Example #5 - More complicated array functions

This example computes the mean-squared power across a vector of $d$ shifted Legendre
polynomials provided as input. This also demonstrates how to probe multiple operators
simultaneously using a fold.

In [ ]:
d = 16
input_functions = [lambda t, p=legendre(i): p(2 * t - 1) for i in range(d)]

stim = gyrus.stimuli(input_functions)
y_hat = (stim ** 2).mean()
y_ideal = stim.apply(np.square).mean()

out_hat, out_ideal = gyrus.fold([y_hat, y_ideal]).filter(5e-3).run(1)

In [ ]:
plt.figure()
plt.title("Example #5 - More complicated array functions")
plt.plot(out_hat, alpha=0.8, label="Spiking")
plt.plot(out_ideal, label="Ideal")
plt.xlabel("Time-step")
plt.legend()
plt.show()

### Example #6 - Building a Nengo network into a Gyrus operator

Suppose we want to define our own Gyrus operator using Nengo code, such that it is
appropriately vectorized across folds. We can do so using the `@gyrus.vectorize`
decorator. Inside of the decorated implementation, we are free to use whatever Nengo
code we'd like, and can even `make()` other Gyrus operators and interface them with the
Nengo code (see example #4). The implementation is expected to return a single Nengo
node.

For sake of example, we'll take an existing Nengo subnetwork
(`nengo.networks.oscillator`) and turn it into a Gyrus operator. We'll also register the
operator as a method on all existing operators using the `@gyrus.register_method`
decorator.

To demonstrate the benefits of doing this, we show that it becomes trivial to
instantiate three different oscillators with three separate kick inputs, simply by
expanding the shape of the input. We can even specify three different frequencies, and
they will be applied element-wise alongside each respective kick. In general, all Gyrus
operators are vectorized and follow the same NumPy semantics as `@np.vectorize` (in
fact, that is what is used under the hood).

Note: A more accurate oscillator implementation is supported by the `lti` method (see
final example #9).

In [ ]:
@gyrus.register_method("oscillate")
@gyrus.vectorize("Oscillate")
def oscillate(node, *, n_neurons, recurrent_tau, frequency, **kwargs):
    oscillator = nengo.networks.Oscillator(
        n_neurons=n_neurons,
        recurrent_tau=recurrent_tau,
        frequency=frequency,
    )
    nengo.Connection(node, oscillator.input, synapse=None)
    return oscillator.output

In [ ]:
strength = np.asarray([0.5, 1, 4])
hz = np.asarray([0.5, 0.75, 1.0])

kicks = gyrus.stimuli([lambda t, r=r: r if t <= 0.05 else 0 for r in strength])
x = kicks.transform([[1], [0]]).oscillate(
    n_neurons=250,
    recurrent_tau=0.1,
    frequency=2 * np.pi * hz,
)
out = np.asarray(x.filter(1e-2).run(1))

In [ ]:
plt.figure(figsize=(5, 5))
plt.title("Example #6 - Building a Nengo network into a Gyrus operator")
for i, xy in enumerate(out):
    plt.plot(*xy.T, label=f"{hz[i]} Hz")
plt.axis("equal")
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.xlabel("$x(t)$")
plt.ylabel("$y(t)$")
plt.legend()
plt.show()

The `gyrus.vectorize` decorator can also be used by directly passing it in an
implementation (rather than a name followed by several optional arguments). This
convenience allows it to be used like a Keras Lambda layer, but for embedding arbitrary
Nengo code into the Gyrus model. For example:

In [ ]:
def multiply_by_two(x):
    y = nengo.Node(size_in=x.size_out)
    nengo.Connection(x, y, transform=2, synapse=None)
    return y


x = gyrus.stimulus(np.ones(3))
y = gyrus.vectorize(multiply_by_two)(x)
y.run(1, dt=1)

### Example #7 - Extending Gyrus to work with existing NumPy functions

Suppose we have some NumPy function that uses `np.linalg.norm`, and we wish to convert
that function into a Nengo network without changing that piece of code. We'll first see
that we get a `TypeError` "no implementation found" when we try to use that function on
a Gyrus operator:

In [ ]:
np.linalg.norm(gyrus.stimuli(0))

To register an implementation, use the `gyrus.register_method` decorator. If one wishes
to override the entire behaviour of `norm`, then one can define:
```python
@gyrus.register_method("norm")
def custom_norm(x, ord=None, axis=None, keepdims=False):
    ...
```
and return a Gyrus operator (or array of operators). This will customize the
implementation of `np.linalg.norm` given an array of operators `x`.

But there is also a much simpler solution in this case; just defer to the underlying
`np.linalg.norm` implementation:

In [ ]:
gyrus.register_method("norm")(np.linalg.norm)

This causes NumPy to invoke its regular routine of squaring, summing, and square-rooting
the results (with appropriate handling of axes), where each element being squared,
summed, and square-rooted is a single Gyrus operator (i.e., an element of the array).
Now, if we try again, we'll get a `TypeError` "all returned `NotImplemented`" -- but
this time because it can't find an implementation for `np.sqrt`.

In [ ]:
np.linalg.norm(gyrus.stimuli(0))

To solve this final problem, we'll need to add an implementation for `np.sqrt`. We do so
using a `@gyrus.register_ufunc(np.sqrt)` decorator which tells NumPy how to handle this
particular ufunc. We'll also use the `@gyrus.vectorize` decorator so that we can provide
the implementation for a single element using Nengo and have it automatically vectorized
across folds (see example #6).

This particular implementation will use an ensemble to decode `np.sqrt(x)` when `x >=
0`, otherwise some `undefined` value for `x < 0` (default `0`). This implementation
could be improved somewhat by specializing the Ensemble's parameters, but this serves as
a straightforward example.

To make this example more complete, we'll also register a number of Ensemble parameters
as `configurable` which allows them to be configured via the `configure` operator.

In [ ]:
from gyrus.nengo_helpers import get_params

configurable = get_params(nengo.Ensemble) - {"dimensions"} | {"seed", "undefined"}
print("Configurable parameters:", configurable)


@gyrus.register_ufunc(np.sqrt)
@gyrus.vectorize("Sqrt", configurable=configurable)
def custom_sqrt(node, *, n_neurons=100, undefined=0, **ens_kwargs):
    """Straightforward implementation of the sqrt nonlinearity."""
    x = nengo.Ensemble(n_neurons, dimensions=node.size_out, **ens_kwargs)
    nengo.Connection(node, x, synapse=None)

    def valid_sqrt(x):
        """Returns sqrt(x) for x >= 0, otherwise undefined."""
        y = np.full_like(x, undefined)
        y[x >= 0] = np.sqrt(x[x >= 0])
        return y

    out = nengo.Node(size_in=node.size_out)
    nengo.Connection(x, out, function=valid_sqrt, synapse=None)
    return out

Note: We could also decorate with `@gyrus.register_method("sqrt")` to attach this as a
`.sqrt()` method to every operator (including folds).

Let's first try this out to see how it looks. To get the ideal output as well, we
configure the same operator in two different ways (this works because operators are
_pure_ functions).

In [ ]:
stim = gyrus.stimuli(lambda t: 2 * t - 1)
x = gyrus.fold(
    [
        stim.configure(n_neurons=250),
        stim.configure(neuron_type=nengo.Direct()),
    ]
)
out_hat, out_ideal = np.sqrt(x).filter(1e-2).run(1)

plt.figure()
plt.title("Example #7 - Extending Gyrus (np.sqrt)")
plt.plot(out_hat, alpha=0.8, label="Spiking")
plt.plot(out_ideal, label="Ideal")
plt.xlabel("Time-step")
plt.legend()
plt.show()

Now `np.linalg.norm` finally works!

In [ ]:
input_functions = [
    lambda t: t * np.sin(2 * np.pi * t),
    lambda t: t * np.cos(2 * np.pi * t),
]
stim = gyrus.stimuli(input_functions)
y_hat = np.linalg.norm(stim)
y_ideal = stim.bundle().apply(np.linalg.norm)
out_hat, out_ideal = gyrus.fold([y_hat, y_ideal]).filter(nengo.Alpha(1e-2)).run(1)

In [ ]:
plt.figure()
plt.title("Example #7 - Extending Gyrus (np.linalg.norm)")
plt.plot(out_hat, alpha=0.8, label="Spiking")
plt.plot(out_ideal, label="Ideal")
plt.xlabel("Time-step")
plt.legend()
plt.show()

Note: Once `np.sqrt` is defined we can also do things like:
```python
y_hat = nengo.utils.numpy.rms(stim)
y_ideal = stim.bundle().apply(nengo.utils.numpy.rms)
```
to implement other existing functions (e.g., `nengo.utils.numpy.rms`) using spikes.

### Example #8 - Integrating Ordinary Differential Equations

Gyrus also supports an `integrate` operator that can be used to integrate arbitrary
differential equations produced by other operators. Here is a simple example launching a
number of parallel integrators in parallel to with cosine inputs (the integral of cosine
is sine).

More generally, an `integrand` function can be supplied to integrate any function of the
integral, `x`, using Gyrus operators (see example #9).

In [ ]:
help(gyrus.integrate)

In [ ]:
d = 32  # number of integrators
dt = 1e-3  # time-step of simulation


def f(t, hz):  # derivative of sinusoid with given frequency
    return np.cos(2 * np.pi * hz * (t - dt)) * (2 * np.pi * hz)

In [ ]:
u = gyrus.stimuli([lambda t, hz=hz: f(t, hz) for hz in np.linspace(1, 2, d)])
x = u.integrate().decode().filter(5e-3)
out = np.asarray(x.run(1, dt=dt))

In [ ]:
colors = sns.color_palette("cubehelix", n_colors=d)

plt.figure()
plt.title("Example #8 - Integrating Ordinary Differential Equations")
for x_i, color in zip(out, colors):
    plt.plot(x_i.squeeze(axis=1), color=color, alpha=0.7)
plt.xlabel("Time-step")
sns.despine(offset=10)
plt.show()

### Example #9 - Improved Oscillator & NengoDL

The `lti` method in Gyrus builds on top of the `integrate` method to solve differential
equations of the form $\dot{x} = A.\text{state}(x) + B.u$. where $A$, $B$ is the desired
system. It does so using zero-order hold (ZOH)—assuming $\text{state}(x) = x$—and
compensates for the simulation time-step. Nonlinear dynamics are supported by specifying
a nonlinear `state(x)` function for the integrand.

Interestingly, the `lti` method is essentially just one line, and thus exists mostly as
a convenience wrapper:
```python
return u.transform(Bmap).integrate(integrand=lambda x: state(x).transform(Amap))
```

For improved accuracy, we also use `unbundle()` and `bundle()` to unbundle the 2D
representation into two 1D representations, and then bundle them back together.

In addition, we use the NengoDL simulator to run this on the GPU, by passing
`simulator=nengo_dl.Simulator` to the `run` method.

In [ ]:
def oscillator(hertz):
    """Linear (A, B) system for an oscillator at given frequency."""
    radians = 2 * np.pi * hertz
    return [[0, -radians], [radians, 0]], [1, 0]

In [ ]:
kick = gyrus.stimuli(lambda t: 1 / dt if t <= dt else 0)


def state(x):
    x = x.configure(neuron_type=nengo.SpikingRectifiedLinear())
    return x.unbundle().decode().bundle()


x = kick.lti(oscillator(hertz=5), state=state, dt=dt)
out = x.run(1, dt=dt, simulator=nengo_dl.Simulator)

In [ ]:
plt.figure()
plt.title("Example #9 - Improved Oscillator & NengoDL")
plt.plot(out)
plt.xlabel("Time-step")
plt.show()

It's also super simple to turn this into a controlled oscillator. Just multiply the
`state` by a `control` signal. One subtle caveat is the ZOH discretization assumes the
dynamics are linear, and so the time-step is no longer ideally accounted for when
`control != 1`. The most accurate approach currently known is a bit more involved:
https://gl.appliedbrainresearch.com/arvoelke/scratchpad/blob/master/benchmarks/oscillator.py#L37

In [ ]:
control = gyrus.stimuli(
    nengo.processes.Piecewise({0: 1, 0.2: -1, 0.4: 0, 0.6: -0.5, 0.8: 0.8})
)


def state(x):
    x = x.configure(n_neurons=500, neuron_type=nengo.LIF())
    return (x.unbundle() * control).bundle()


x = kick.lti(oscillator(hertz=5), state=state, dt=dt)
out_x, out_control = gyrus.fold([x, control]).run(
    1, dt=dt, simulator=nengo_dl.Simulator
)

In [ ]:
plt.figure()
plt.title("Example #9 - Improved Oscillator & NengoDL (Controlled)")
plt.plot(out_x, label="$x(t)$")
plt.plot(out_control, linestyle="--", label="Control")
plt.xlabel("Time-step")
plt.legend()
plt.show()

There's still plenty more that can be done with the Gyrus API that is not shown here.
Check out the rest of the examples in this directory!